# Laboratorio 1
## Detección de phishing en URL

*Phishing*<br>
Se basa en la ingeniería social (manipulación de emociones, aprovechamiento de atajos mentales y
sesgos cognitivos) para engañar a las víctimas y lograr que estas den información (normalmente
credenciales). Los atacantes envían mensajes haciéndose pasar por una entidad legitima a través de
correos y SMS bajo diversos “motivos urgentes” que requieren que la persona tome acción
inmediatamente, para lo cual incluyen un enlace que redirige al “sitio web” de la entidad.<br>

Estos sitios son literalmente copias de los sitios legítimos que intentan imitar, en muchas ocasiones
son muy difíciles de detectar. El usuario, temeroso de un evento negativo ingresa con sus credenciales,
las cuales son robadas y utilizadas por los atacantes para acceder a los verdaderos sitios legítimos
ocasionando pérdidas económicas (entre otros).<br>

Sin embargo, los dominios web no pueden copiarse al 100%, aunque existen técnicas avanzadas que
los hacen parecer similares al ojo humano. Además, las URLs de phishing poseen características que
las diferencian de las URLs legítimas, y que un modelo de ML puede utilizar para detectarlas y proteger
a los usuarios de estos ataques.

### Parte 1: Ingeniería de características
*Exploración de datos*

Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones.

In [17]:
import pandas as pd
import os 


directory: str = os.getcwd()
df: pd.DataFrame = pd.read_csv(f"{directory}/dataset_pishing.csv")


In [18]:
df.head(5)

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate


Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como
“pishing”. ¿Está balanceado el dataset?

Sí está balanceado el conjunto de datos ya que tiene 5715 legítimos y el mismo número para phishing.

In [19]:
df['status'].value_counts()

status
legitimate    5715
phishing      5715
Name: count, dtype: int64

*Derivación de características*<br>
Revise los artículos proporcionados, especialmente en el análisis de las URLs. En base a su análisis
responda las siguientes preguntas:<br>
1. ¿Qué ventajas tiene el análisis de una URL contra el análisis de otros datos, cómo el tiempo
de vida del dominio, o las características de la página Web? <br>
El análisis de una URL tiene como ventaja que podemos detectar sitios maliciosos y de una vez bloquear su dirección IP para prevenir que se pueda acceder a 
otras páginas web dentro de la misma. También que se puede tomar cierta independencia del lenguaje en el cual está escrita la página. Y de igual forma que necesitaría menos características
para entrenar un modelo. 
2. ¿Qué características de una URL son más prometedoras para la detección de phishing?<br>
El protocolo que utiliza la url para la comunicación, la longitud de la url, cierto tipo de símbolos aparecidos dentro de la misma, por ejemplo: comas, slashes, guiones, etc. Tabmién si y emails presentes en las url y los subdominios, dominios y los supradominios. 

In [ ]:
import re
from urllib.parse import urlparse

# Base functions
def get_protocol(url: str) -> str:
    protocol: str = url.split(':')[0]
    return protocol

def get_subdomain(url: str) -> str:
    parsed = urlparse(url)
    parts = parsed.hostname.split('.') if parsed.hostname else []
    return '.'.join(parts[:-2]) if len(parts) > 2 else 'missing'

def get_second_level_domain(url: str) -> str:
    parsed = urlparse(url)
    parts = parsed.hostname.split('.') if parsed.hostname else []
    return parts[-2] if len(parts) >= 2 else 'missing'


def get_top_level_domain(url: str) -> str:
    parsed = urlparse(url)
    parts = parsed.hostname.split('.') if parsed.hostname else []
    return parts[-1] if len(parts) >= 1 else 'missing'


def has_file(url: str) -> int:
    parsed = urlparse(url)
    if parsed.path:
        if parsed.path.split('/')[-1] != '':
            return 1
    return 0

def has_parameters(url: str) -> int:
    parsed = urlparse(url)
    if parsed.query != '':
        return 1
    return 0

def email_in_url(url: str) -> int:
    return int(bool(re.search(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', url)))

def has_ip(url: str) -> int:
    '''If the URL has an IP address'''
    return int(bool(re.search(r'https?://(?:\d{1,3}\.){3}\d{1,3}', url)))

def length_url(url: str) -> int:
    return len(url)

# helper
def count_symbol(url: str, symbol: str) -> int:
    if url:
        return url.count(symbol)
    return 0

def num_dots_url(url: str) -> int:
    return count_symbol(url=url, symbol='.')

def num_hyph_url(url: str) -> int:
    return count_symbol(url=url, symbol="-")

def num_underline_url(url: str) -> int:
    return count_symbol(url=url, symbol="_")

def num_slash_url(url: str) -> int:
    return count_symbol(url=url, symbol="/")

def at_sign_url(url: str) -> int:
    return count_symbol(url=url, symbol="@")

def num_plus_url(url: str) -> int:
    return count_symbol(url=url, symbol="+")

def num_and_symbols(url: str) -> int:
    return count_symbol(url=url, symbol="&")

def num_equal_url(url: str) -> int:
    return count_symbol(url=url, symbol="=")

def num_question_url(url: str) -> int:
    return count_symbol(url=url, symbol="?")

In [ ]:
import math
from collections import Counter

def count_dict(url: str) -> dict:
    counter = Counter(url)
    return dict(counter)


def shannon_entropy(url: str) -> float:
    entropy: float = 0.0
    length: int = len(url)
    frequencies: dict = count_dict(url)
    for _, freq in frequencies.items():

        probability: float = freq/length
        if probability == 0:
            continue
        entropy += probability * math.log2(1/probability)
    return entropy

def relative_entropy(url: str) -> float:
    entropy: float = 0.0
    frequencies: dict = count_dict(url)
    for letter, freq in frequencies.items():
        probability: float = freq
        if probability == 0:
            continue
        q_prob: float = Q.get(letter, 1e-6)
        entropy += probability * math.log(probability / q_prob)
    return entropy

In [21]:
df['protocol'] = df['url'].apply(get_protocol) 
df['subdomain'] = df['url'].apply(get_subdomain)
df['second_domain'] = df['url'].apply(get_second_level_domain)
df['top_domain'] = df['url'].apply(get_top_level_domain)
df['file_name'] = df['url'].apply(has_file)
df['parameters'] = df['url'].apply(has_parameters)
df['email'] = df['url'].apply(email_in_url)
df['ip'] = df['url'].apply(has_ip)
df['length'] = df['url'].apply(length_url)
df['num_dots'] = df['url'].apply(num_dots_url)
df['num_hyph'] = df['url'].apply(num_hyph_url)
df['num_underline'] = df['url'].apply(num_underline_url)
df['num_slash'] = df['url'].apply(num_slash_url)
df['num_at_sign'] = df['url'].apply(at_sign_url)
df['num_plus'] = df['url'].apply(num_plus_url)
df['num_and'] = df['url'].apply(num_and_symbols)
df['num_equal'] = df['url'].apply(num_equal_url)
df['num_question'] = df['url'].apply(num_question_url)

*Preprocesamiento*<br>

Modificar la variable categórica *status* a una variable binaria. <br>
```
legitimate -> 0
phishing -> 1
```

In [22]:
df['status'] = df['status'].map({'legitimate' : 0, 'phishing' : 1})
df.head(5)

,url,status,protocol,subdomain,second_domain,top_domain,file_name,parameters,email,ip,length,num_dots,num_hyph,num_underline,num_slash,num_at_sign,num_plus,num_and,num_equal,num_question
0,http://www.crestonwood.com/router.php,0,http,www,crestonwood,com,1,0,0,0,37,3,0,0,3,0,0,0,0,0
1,http://shadetreetechnology.com/V4/validation/a...,1,http,missing,shadetreetechnology,com,1,0,0,0,77,1,0,0,5,0,0,0,0,0
2,https://support-appleld.com.secureupdate.duila...,1,https,support-appleld.com.secureupdate,duilawyeryork,com,0,1,0,0,126,4,1,2,5,0,0,2,3,1
3,http://rgipt.ac.in,0,http,rgipt,ac,in,0,0,0,0,18,2,0,0,2,0,0,0,0,0
4,http://www.iracing.com/tracks/gateway-motorspo...,0,http,www,iracing,com,0,0,0,0,55,2,2,0,5,0,0,0,0,0


In [23]:
df = df.drop(columns=['url'])

In [24]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   status         11430 non-null  int64
 1   protocol       11430 non-null  str  
 2   subdomain      11430 non-null  str  
 3   second_domain  11430 non-null  str  
 4   top_domain     11430 non-null  str  
 5   file_name      11430 non-null  int64
 6   parameters     11430 non-null  int64
 7   email          11430 non-null  int64
 8   ip             11430 non-null  int64
 9   length         11430 non-null  int64
 10  num_dots       11430 non-null  int64
 11  num_hyph       11430 non-null  int64
 12  num_underline  11430 non-null  int64
 13  num_slash      11430 non-null  int64
 14  num_at_sign    11430 non-null  int64
 15  num_plus       11430 non-null  int64
 16  num_and        11430 non-null  int64
 17  num_equal      11430 non-null  int64
 18  num_question   11430 non-null  int64
dtypes: int64(15), s

*Selección de características*

En la exploración de datos, determine las columnas que son constantes, o que no tienen una varianza
alta con la columna status. Elimine las características repetidas o irrelevantes para la clasificación de
un sitio de phishing. Verifique que no posee observaciones repetidas. Apóyese con la visualización de
características y correlación para seleccionar las características más importantes para clasificar una
URL legítima de una URL de phishing. <br>

¿Qué columnas o características fueron seleccionadas y por qué?<br>
Las columnas seleccionadas fueron las siguientes ... 

### Parte 2: Implementación

*Separación de datos*
- Datos de entrenamiento: 55%
- Datos de validación: 15%
- Datos de prueba: 30%
- Almacene cada dataset como un archivo .csv

In [ ]:
# TODO separate data

*Implementación*
Implemente dos modelos de Machine Learning (a su discreción) para la clasificación de phishing.
Muestre y explique los valores obtenidos de las siguientes métricas para los datos de validación y
pruebas, para cada modelo, en base al contexto del problema (detección de Pishing).
- Matriz de confusión
- Precision
- Recall
- Curva ROC
- AUC

In [ ]:
# TODO implement Random Forest Classifier

In [ ]:
# TODO implement Naïve Bayes

*Discusión*

4. ¿Cuál es el impacto de clasificar un sitio legítimo como phishing?
5. ¿Cuál es el impacto de clasificar un sitio de phishing como legítimo?
6. En base a las respuestas anteriores, ¿Qué métrica elegiría para comparar modelos similares
de clasificación de phishing?
7. ¿Qué modelo funcionó mejor para la clasificación de phishing? ¿Por qué?
8. Una empresa desea utilizar su mejor modelo, debido a que sus empleados sufren constantes
ataques de phishing mediante e-mail. La empresa estima que, de un total de 50,000 emails,
un 15% son phishing. ¿Qué cantidad de alarmas generaría su modelo? ¿Cuántas positivas y
cuantas negativas? ¿Funciona el modelo para el BR propuesto? En caso negativo, ¿qué
propone para reducir la cantidad de falsas alarmas?